# Neural Approaches to Recommendation Systems

Recommender Systems are one of the most popular applications of Machine Learning systems. Due to their widespread success, they are quickly becoming ubiquitous to a lot of businesses. Traditionally, collaborative filtering and matrix factorization techniques were used to solve these problems.

In the last couple of years, this trend has been changing. Due to the massive success of effectively training deep neural nets, new approaches have been developed by leveraging the tools and modeling flexibility from the Deep Learning ecosystem.

This hack session gives a primer into these concepts using neural network architectures.

For those who are interested in an intuitive explanation to collaborative filtering and embeddings, please refer to this brilliant **fast.ai lesson by Jeremy and Rachel - http://course.fast.ai/lessons/lesson4.html**

Recommendation Engines
============

## Common Applications
    - Feed (News Feed on Facebook, Feed on Twitter, Explore on Instagram, Home Page of Amazon etc.)
    - Rule of thumb: 
        - Large item inventory
        - Thus discoverability is a problem.
        - Recsys will make their mark.
    - Traditional methods in recsys
        - User - User similarity
        - Item - Item similarity
        - Hybrid models - Collaborative Filtering
        - Matrix factorization
    - Entry of Neural Approaches
        - Latent factors in earlier approaches analogous to Embeddings in deep learning ecosystem
        - GPU training, superior optimization techniques (Adam etc.) enabled neural approaches to be preferred.
        - Flexibility of adding layers, ease of adding additional metadata and joint-training is a plus in this approach.


### Let's dive!

Table of Contents:
    0. Installations
    1. Import the necessary libraries (print versions of the libraries)
    2. Read the necessary datasets
    3. Create the interactions frame
    4. Split the frame into train and validation sets
    5. Create the keras network (after creating necessary embeddings)
    6. Train the network and monitor accuracy on validation
    7. Make the network deeper by adding dense layers and re-train the network

# Installations
    - wget https://repo.continuum.io/archive/Anaconda3-5.0.0-MacOSX-x86_64.sh # Anaconda Python 3.6 installer
    - conda install -c conda-forge keras # Install Keras

# 1. Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np

# Setting seed before importing keras to ensure reproducibility
np.random.seed(2017)
import keras as K

print("Pandas version: ", pd.__version__)
print("Numpy version: ", np.__version__)
print("Keras version: ", K.__version__)


Using Theano backend.


('Pandas version: ', u'0.18.1')
('Numpy version: ', '1.11.1')
('Keras version: ', '1.1.0')


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# 2. Read the necessary datasets

In [2]:
# Reading in the datasets
train = pd.read_csv("../input/train.csv"); print(train.shape)
test = pd.read_csv("../input/test.csv"); print(test.shape)

print("# Users: {} | # Articles: {}".format(train.User_ID.nunique(), train.Article_ID.nunique()))

diff = np.setdiff1d(train.User_ID.unique(), test.User_ID.unique())
train = train[~train.User_ID.isin(diff)].reset_index(drop=True) # Drop train-only users.

(679051, 4)
(291023, 3)
# Users: 73489 | # Articles: 214027


# 3. Create the interactions frame

In [3]:
# Creating one dataframe of the interactions
ratings = pd.concat([train, test])

users = ratings.User_ID.unique()
articles = ratings.Article_ID.unique()

# Create userid & itemid to index mappings
userid2idx = {o:i for i,o in enumerate(users)}
articlesid2idx = {o:i for i,o in enumerate(articles)}

ratings.Article_ID = ratings.Article_ID.apply(lambda x: articlesid2idx[x])
ratings.User_ID = ratings.User_ID.apply(lambda x: userid2idx[x])

n_users = ratings.User_ID.nunique()
n_articles = ratings.Article_ID.nunique()

# 4. Split the frame into train and validation sets

In [4]:
X_train = ratings[0:len(train)]; print(X_train.shape)
X_test = ratings[len(train):len(ratings)]; print(X_test.shape)

# Split the data into train and validation sets
np.random.seed(2017)
msk = np.random.rand(len(X_train)) < 0.8
trn = X_train[msk]
val = X_train[~msk]

(618437, 4)
(291023, 4)


# 5. Create the keras network 
    - After creating necessary embeddings for each User_ID and Article_ID

In [7]:
n_factors = 50
import keras.backend as K

def rmse(y_true, y_pred):
    score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
    return score

from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import Callback, TensorBoard

def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
article_in, a = embedding_input('article_in', n_articles, n_factors, 1e-5)

ub = create_bias(user_in, n_users)
ab = create_bias(article_in, n_articles)

x = merge([u, a], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, ab], mode='sum')

model = Model([user_in, article_in], x)
model.compile(Adam(5e-3), loss='mse', metrics=[rmse])

# 6. Train the network and monitor accuracy on validation

In [8]:
model.fit([trn.User_ID, trn.Article_ID], trn.Rating,
          nb_epoch=10, batch_size=1024,
          validation_data=([val.User_ID, val.Article_ID], val.Rating))

Train on 494544 samples, validate on 123893 samples
Epoch 1/10
494544/494544 [==============================] - 25s - loss: 5.1717 - rmse: 2.2598 - val_loss: 4.6196 - val_rmse: 2.1485
Epoch 2/10
494544/494544 [==============================] - 25s - loss: 4.1626 - rmse: 1.9570 - val_loss: 4.0984 - val_rmse: 2.0236
Epoch 3/10
494544/494544 [==============================] - 25s - loss: 3.0437 - rmse: 1.5325 - val_loss: 3.9732 - val_rmse: 1.9925
Epoch 4/10
494544/494544 [==============================] - 25s - loss: 2.4167 - rmse: 1.2179 - val_loss: 3.9204 - val_rmse: 1.9792
Epoch 5/10
494544/494544 [==============================] - 25s - loss: 2.1524 - rmse: 1.0553 - val_loss: 3.8827 - val_rmse: 1.9696
Epoch 6/10
494544/494544 [==============================] - 25s - loss: 2.0053 - rmse: 0.9676 - val_loss: 3.8420 - val_rmse: 1.9593
Epoch 7/10
494544/494544 [==============================] - 25s - loss: 1.9005 - rmse: 0.9141 - val_loss: 3.8089 - val_rmse: 1.9508
Epoch 8/10
494544/494544

# 7. Make the network deeper by adding dense layers and re-train the network

In [14]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
article_in, a = embedding_input('article_in', n_articles, n_factors, 1e-5)

ub = create_bias(user_in, n_users)
ab = create_bias(article_in, n_articles)

x = merge([u, a], mode='concat')
x = Flatten()(x)

# Dense connections
x = Dropout(0.5)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)

model = Model([user_in, article_in], x)
model.compile(Adam(5e-3), loss='mse', metrics=[rmse])

model.fit([trn.User_ID, trn.Article_ID], trn.Rating,
          nb_epoch=10, batch_size=2048,
          validation_data=([val.User_ID, val.Article_ID], val.Rating))

Train on 494544 samples, validate on 123893 samples
Epoch 1/10
494544/494544 [==============================] - 12s - loss: 3.8499 - rmse: 1.9420 - val_loss: 3.3099 - val_rmse: 1.8189
Epoch 2/10
494544/494544 [==============================] - 12s - loss: 3.4671 - rmse: 1.8049 - val_loss: 3.3192 - val_rmse: 1.8215
Epoch 3/10
494544/494544 [==============================] - 12s - loss: 3.2863 - rmse: 1.7276 - val_loss: 3.3685 - val_rmse: 1.8350
Epoch 4/10
494544/494544 [==============================] - 12s - loss: 3.2012 - rmse: 1.6841 - val_loss: 3.4301 - val_rmse: 1.8516
Epoch 5/10
494544/494544 [==============================] - 12s - loss: 3.1671 - rmse: 1.6602 - val_loss: 3.4676 - val_rmse: 1.8617
Epoch 6/10
494544/494544 [==============================] - 12s - loss: 3.1606 - rmse: 1.6490 - val_loss: 3.4886 - val_rmse: 1.8674
Epoch 7/10
494544/494544 [==============================] - 12s - loss: 3.1543 - rmse: 1.6411 - val_loss: 3.4903 - val_rmse: 1.8678
Epoch 8/10
494544/494544

In [ ]:
# 2048 - 13s - loss: 2.9813 - rmse: 1.5600 - val_loss: 3.5050 - val_rmse: 1.8718
# 1024 - 25s - loss: 3.2050 - rmse: 1.6015 - val_loss: 3.4731 - val_rmse: 1.8629

# Using side information

Often, along with the user-interaction data, other information such as user metadata and item metadata is also given. With the above networks, it's trivial to add this metadata to our model. Let's see how.

In [34]:
# 2. Read the necessary datasets
user = pd.read_csv("../input/user.csv")
article = pd.read_csv("../input/article.csv")
train = pd.read_csv("../input/train.csv"); print(train.shape)
test = pd.read_csv("../input/test.csv"); print(test.shape)

diff = np.setdiff1d(train.User_ID.unique(), test.User_ID.unique())
train = train[~train.User_ID.isin(diff)].reset_index(drop=True) # Drop train-only users.

train = train.merge(user, how='left'); print(train.shape)
train = train.merge(article, how='left'); print(train.shape)
test = test.merge(user, how='left'); print(test.shape)
test = test.merge(article, how='left'); print(test.shape)



# For simplicity, impute with 0.
# Ideally, you should either do mean / median imputation for numeric vars & mode imputation for cat vars.
train = train.fillna(0)
test = test.fillna(0)


# 3. Create the interactions frame
ratings = pd.concat([train, test])

# Scaling numeric columns
from sklearn.preprocessing import scale
ratings.VintageMonths = scale(ratings.VintageMonths)

users = ratings.User_ID.unique()
articles = ratings.Article_ID.unique()
age = ratings.Age.unique()
var1 = ratings.Var1.unique()

# Create userid & itemid to index mappings
userid2idx = {o:i for i,o in enumerate(users)}
articlesid2idx = {o:i for i,o in enumerate(articles)}
age2idx = {o:i for i,o in enumerate(age)}
var12idx = {o:i for i,o in enumerate(var1)}

ratings.Article_ID = ratings.Article_ID.apply(lambda x: articlesid2idx[x])
ratings.User_ID = ratings.User_ID.apply(lambda x: userid2idx[x])
ratings.Age = ratings.Age.apply(lambda x: age2idx[x])
ratings.Var1 = ratings.Var1.apply(lambda x: var12idx[x])

n_users = ratings.User_ID.nunique()
n_articles = ratings.Article_ID.nunique()
n_age = ratings.Age.nunique()
n_var1 = ratings.Var1.nunique()


# 4. Split the frame into train and validation sets
X_train = ratings[0:len(train)]; print(X_train.shape)
X_test = ratings[len(train):len(ratings)]; print(X_test.shape)

# Split the data into train and validation sets
np.random.seed(2017)
msk = np.random.rand(len(X_train)) < 0.8
trn = X_train[msk]
val = X_train[~msk]

(679051, 4)
(291023, 3)
(618437, 6)
(618437, 9)
(291023, 5)
(291023, 8)
(618437, 9)
(291023, 9)


## Add age and other numeric variables

In [43]:
# TODO
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
article_in, a = embedding_input('article_in', n_articles, n_factors, 1e-5)

meta_input_f0 = Input(shape=[1], name='meta_input_f0') # Age
meta_input_f1 = Input(shape=[1], name='meta_input_f1') # NumberOfArticlesBySameAuthor
meta_input_f2 = Input(shape=[1], name='meta_input_f2') # NumberOfArticlesinSameCategory
meta_input_f3 = Input(shape=[1], name='meta_input_f3') # VintageMonths
meta_input_f4 = Input(shape=[1], name='meta_input_f4') # Var1

ub = create_bias(user_in, n_users)
ab = create_bias(article_in, n_articles)

x = merge([u, a], mode='concat')
x = Flatten()(x)

# Dense connections
x = Dropout(0.2)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1)(x)

model = Model([user_in, article_in, meta_input_f0, meta_input_f1, meta_input_f2, meta_input_f3, meta_input_f4], x)
model.compile(Adam(5e-3), loss='mse', metrics=[rmse])

model.fit([trn.User_ID, trn.Article_ID, trn.Age, trn['NumberOfArticlesBySameAuthor\r'], trn['NumberOfArticlesinSameCategory\r'], trn['VintageMonths'], trn['Var1']], trn.Rating,
          nb_epoch=10, batch_size=2048,
          validation_data=([val.User_ID, val.Article_ID, val.Age, val['NumberOfArticlesBySameAuthor\r'], val['NumberOfArticlesinSameCategory\r'], val['VintageMonths'], val['Var1']], val.Rating))

Train on 494544 samples, validate on 123893 samples
Epoch 1/10
494544/494544 [==============================] - 13s - loss: 3.6028 - rmse: 1.8727 - val_loss: 3.2393 - val_rmse: 1.7994
Epoch 2/10
494544/494544 [==============================] - 13s - loss: 3.1278 - rmse: 1.6928 - val_loss: 3.3074 - val_rmse: 1.8182
Epoch 3/10
494544/494544 [==============================] - 13s - loss: 2.7962 - rmse: 1.5533 - val_loss: 3.4844 - val_rmse: 1.8662
Epoch 4/10
494544/494544 [==============================] - 13s - loss: 2.5877 - rmse: 1.4673 - val_loss: 3.5905 - val_rmse: 1.8945
Epoch 5/10
494544/494544 [==============================] - 13s - loss: 2.4664 - rmse: 1.4096 - val_loss: 3.7130 - val_rmse: 1.9265
Epoch 6/10
494544/494544 [==============================] - 13s - loss: 2.3786 - rmse: 1.3625 - val_loss: 3.7056 - val_rmse: 1.9246
Epoch 7/10
494544/494544 [==============================] - 13s - loss: 2.3156 - rmse: 1.3253 - val_loss: 3.7644 - val_rmse: 1.9398
Epoch 8/10
494544/494544

In [ ]:
# 13s - loss: 3.7780 - rmse: 1.9228 - val_loss: 3.2853 - val_rmse: 1.8121

In [47]:
# TODO
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
article_in, a = embedding_input('article_in', n_articles, n_factors, 1e-5)

meta_input_f0 = Input(shape=[1], name='meta_input_f0') # Age
meta_input_f1 = Input(shape=[1], name='meta_input_f1') # NumberOfArticlesBySameAuthor
meta_input_f2 = Input(shape=[1], name='meta_input_f2') # NumberOfArticlesinSameCategory
meta_input_f3 = Input(shape=[1], name='meta_input_f3') # VintageMonths
meta_input_f4 = Input(shape=[1], name='meta_input_f4') # Var1

ub = create_bias(user_in, n_users)
ab = create_bias(article_in, n_articles)

x = merge([Flatten()(u), Flatten()(a), meta_input_f0], mode='concat')
# x = Flatten()(x)

# Dense connections
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)

model = Model([user_in, article_in, meta_input_f0], x)
model.compile(Adam(5e-3), loss='mse', metrics=[rmse])

model.fit([trn.User_ID, trn.Article_ID, trn.Age], trn.Rating,
          nb_epoch=10, batch_size=2048,
          validation_data=([val.User_ID, val.Article_ID, val.Age], val.Rating))

Train on 494544 samples, validate on 123893 samples
Epoch 1/10
494544/494544 [==============================] - 13s - loss: 3.7576 - rmse: 1.9179 - val_loss: 3.2908 - val_rmse: 1.8136
Epoch 2/10
494544/494544 [==============================] - 12s - loss: 3.3547 - rmse: 1.7715 - val_loss: 3.2690 - val_rmse: 1.8076
Epoch 3/10
494544/494544 [==============================] - 13s - loss: 3.1504 - rmse: 1.6815 - val_loss: 3.3650 - val_rmse: 1.8340
Epoch 4/10
494544/494544 [==============================] - 12s - loss: 3.0426 - rmse: 1.6260 - val_loss: 3.4146 - val_rmse: 1.8475
Epoch 5/10
494544/494544 [==============================] - 12s - loss: 2.9964 - rmse: 1.5957 - val_loss: 3.5024 - val_rmse: 1.8710
Epoch 6/10
494544/494544 [==============================] - 13s - loss: 2.9748 - rmse: 1.5783 - val_loss: 3.4866 - val_rmse: 1.8668
Epoch 7/10
494544/494544 [==============================] - 13s - loss: 2.9676 - rmse: 1.5691 - val_loss: 3.4986 - val_rmse: 1.8700
Epoch 8/10
494544/494544

# Credit where it's due
    - A brilliant fast.ai course by Jeremy and Rachel. Refer to Lesson 4 for Collaborative Filtering lecture.
    - https://m2dsupsdlclass.github.io/lectures-labs/
    - Deep Recommender models using PyTorch - https://github.com/maciejkula/spotlight
    - YouTube Recommendation Engine - http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf
    - Keras Merge Layer - https://faroit.github.io/keras-docs/1.0.4/getting-started/sequential-model-guide/